# Introduction to Data Science

# The CRISP-DM approach to Data Science Tasks

_________________________

# 1. Business Understanding
                    
### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives:  

+ Assess situation:  

+ Determine data mining goals:  

+ Produce project plan:  



In [7]:
import pandas as pd

# aquisição dos dados
import basedosdados as bd

# visualização
from matplotlib import pyplot as plt
import seaborn as sns

# 2. Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

## 2.a. Collect initial data:  

In [8]:
# Define algumas variáveis úteis
dataset_id = "br_inep_indicadores_educacionais"
billing_project_id="trabalho-amd"

Agora visualizamos a estrutura do dataset

In [9]:
bd.list_dataset_tables(dataset_id=dataset_id)


table_id: 
	brasil 
-----------------------------------------------------------------------------------------------
table_id: 
	brasil_remuneracao_docentes 
-----------------------------------------------------------------------------------------------
table_id: 
	escola 
-----------------------------------------------------------------------------------------------
table_id: 
	escola_nivel_socioeconomico 
-----------------------------------------------------------------------------------------------
table_id: 
	fluxo_educacao_superior 
-----------------------------------------------------------------------------------------------
table_id: 
	municipio 
-----------------------------------------------------------------------------------------------
table_id: 
	municipio_taxas_transicao 
-----------------------------------------------------------------------------------------------
table_id: 
	regiao 
--------------------------------------------------------------------------------------

Download das tabelas do dataset:
(o código comentado baixa do site os .csv)

In [ ]:
#bd.download(dataset_id=dataset_id, table_id='escola', savepath='data/escola.csv', billing_project_id=billing_project_id)
df_escola = pd.read_csv('data/escola.csv')

#bd.download(dataset_id=dataset_id, table_id='escola_nivel_socioeconomico', savepath='data/escola_nivel_socioeconomico.csv', billing_project_id=billing_project_id)
df_escola_nivel_socioeconomico = pd.read_csv('data/escola_nivel_socioeconomico.csv')

In [35]:
#bd.download(dataset_id=dataset_id, table_id='brasil', savepath='data/brasil.csv', billing_project_id=billing_project_id)
df_brasil = pd.read_csv('data/brasil.csv')

#bd.download(dataset_id=dataset_id, table_id='brasil_remuneracao_docentes', savepath='data/brasil_remuneracao_docentes.csv', billing_project_id=billing_project_id)
df_brasil_remuneracao = pd.read_csv('data/brasil_remuneracao_docentes.csv')

#bd.download(dataset_id=dataset_id, table_id='municipio', savepath='data/municipio.csv', billing_project_id=billing_project_id)
df_municipio = pd.read_csv('data/municipio.csv')

#bd.download(dataset_id=dataset_id, table_id='municipio_taxas_transicao', savepath='data/municipio_taxas_transicao.csv', billing_project_id=billing_project_id)
df_municipio_taxas = pd.read_csv('data/municipio_taxas_transicao.csv')

#bd.download(dataset_id=dataset_id, table_id='regiao', savepath='data/regiao.csv', billing_project_id=billing_project_id)
df_regiao = pd.read_csv('data/regiao.csv')

#bd.download(dataset_id=dataset_id, table_id='uf', savepath='data/uf.csv', billing_project_id=billing_project_id)
df_uf = pd.read_csv('data/uf.csv')

#bd.download(dataset_id=dataset_id, table_id='uf_remuneracao_docentes', savepath='data/uf_remuneracao_docentes.csv', billing_project_id=billing_project_id)
df_uf_remuneracao = pd.read_csv('data/uf_remuneracao_docentes.csv')

## 2.b. Describe Data

In [12]:
df_escola = pd.read_csv('data/escola.csv')

Começando pela tabela `escola` que é a mais importante do dataset, podemos ver que ela é formada por 208 colunas, destas 205 numéricas e 3 categóricas.

Algumas colunas notáveis são:
+ **rede** (municipal, estadual, federal, privada)
+ **localização** (urbana, rural)
+ **atu**: Média de Alunos por Turma
+ **had**: Média de Horas-Aula diária
+ **tdi**: Taxa de distorção idade-série
+ **taxa_aprovação/taxa_reprovação**
+ **taxa_abandono**
+ **tnr**: Taxa de não resposta
+ **dsu**: Percentual de Docentes com Curso Superior
+ **afd**: Adequação da Formação Docente
+ **ird_media_regularidade_docente**: Regularidade do corpo docente nas escolas de educação básica

Explicação de cada índice:
http://www.gestaouniversitaria.com.br/artigos/conheca-os-indicadores-que-detalham-as-tendencias-da-educacao-basica-brasileira

In [13]:
df_escola.info()
df_escola.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637557 entries, 0 to 2637556
Columns: 208 entries, ano to icg_nivel_complexidade_gestao_escola
dtypes: float64(202), int64(3), object(3)
memory usage: 4.1+ GB


,ano,id_municipio,rede,id_escola,localizacao,atu_ei,atu_ei_creche,atu_ei_pre_escola,atu_ef,atu_ef_anos_iniciais,...,ied_ef_anos_finais_nivel_4,ied_ef_anos_finais_nivel_5,ied_ef_anos_finais_nivel_6,ied_em_nivel_1,ied_em_nivel_2,ied_em_nivel_3,ied_em_nivel_4,ied_em_nivel_5,ied_em_nivel_6,icg_nivel_complexidade_gestao_escola
0,2013,1100015,municipal,11024542,rural,NaN,NaN,NaN,5.0,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 1
1,2013,1100015,municipal,11025000,rural,NaN,NaN,NaN,6.0,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 1
2,2013,1100015,municipal,11025115,rural,11.0,NaN,11.0,13.0,NaN,...,30.8,7.7,7.7,NaN,NaN,NaN,NaN,NaN,NaN,nivel 2
3,2013,1100015,municipal,11047461,urbana,19.3,NaN,19.3,20.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nivel 2
4,2013,1100023,municipal,11007850,rural,NaN,NaN,NaN,18.1,17.2,...,71.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 3
5,2013,1100023,estadual,11048620,urbana,NaN,NaN,NaN,29.9,27.8,...,33.3,44.4,5.6,0.0,0.0,0.0,46.6,46.7,6.7,nivel 6
6,2013,1100023,municipal,11007931,urbana,26.1,NaN,26.1,27.7,27.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nivel 5
7,2013,1100049,municipal,11026294,rural,13.0,NaN,13.0,17.1,17.6,...,42.9,0.0,14.3,NaN,NaN,NaN,NaN,NaN,NaN,nivel 2
8,2013,1100049,estadual,11026014,urbana,NaN,NaN,NaN,19.0,19.8,...,71.4,14.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 3
9,2013,1100049,estadual,11026545,urbana,NaN,NaN,NaN,27.5,27.4,...,33.3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 3


### Explore Data

# 3. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

A ideia é apenas trabalhar com os dados do ensino médio,portanto filtrando as colunas, temos:

In [33]:
# seleciona as colunas relacionadas ao EM:
em_cols = ['ano', 'id_municipio', 'rede', 'id_escola', 'localizacao']
for col in df_escola.columns:
    # se a coluna tem a string "em" seleciona ela
    if "em" in col:
        em_cols.append(col)

# redefine o dataset apenas com essas colunas
df_escola_em = df_escola[em_cols]

# remove as linhas que não tem dados do EM (consideramos que se não há taxa de aprovação, a escola não tem EM)
df_escola_em = df_escola_em.dropna(subset=['taxa_aprovacao_em'])

E aqui temos o dataset filtrado:

In [34]:
df_escola_em.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350586 entries, 5 to 2637554
Data columns (total 58 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ano                             350586 non-null  int64  
 1   id_municipio                    350586 non-null  int64  
 2   rede                            350586 non-null  object 
 3   id_escola                       350586 non-null  int64  
 4   localizacao                     350586 non-null  object 
 5   atu_em                          350062 non-null  float64
 6   atu_em_1_ano                    342918 non-null  float64
 7   atu_em_2_ano                    336292 non-null  float64
 8   atu_em_3_ano                    329100 non-null  float64
 9   atu_em_4_ano                    16415 non-null   float64
 10  atu_em_nao_seriado              8344 non-null    float64
 11  had_em                          276085 non-null  float64
 12  had_em_1_ano   

Agora para a tabela `escola_nivel_socioeconômico`:

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 